In [42]:
import os

import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path

In [2]:
# municipal boundaries

boundaries = gpd.read_file(Path(".") / "open_toronto_ca" / "Former Municipality Boundaries Data.geojson")
boundaries[["_id", "AREA_NAME", "geometry"]]

,_id,AREA_NAME,geometry
0,1,YORK,"MULTIPOLYGON (((-79.49262 43.64744, -79.49249 ..."
1,2,NORTH YORK,"MULTIPOLYGON (((-79.53584 43.70812, -79.53591 ..."
2,3,EAST YORK,"MULTIPOLYGON (((-79.28776 43.68978, -79.28909 ..."
3,4,SCARBOROUGH,"MULTIPOLYGON (((-79.28776 43.68978, -79.28792 ..."
4,5,ETOBICOKE,"MULTIPOLYGON (((-79.48165 43.61872, -79.48180 ..."
5,6,TORONTO,"MULTIPOLYGON (((-79.39652 43.62232, -79.39685 ..."


In [3]:
boundaries._id.dtype

dtype('int64')

In [4]:
# banner 1
banner1 = pd.read_csv(Path(".") / "Toronto Foundation" / "social_capital_2022_banner_1.csv", encoding="windows-1252")

In [5]:
banner1_geography = banner1[banner1["category"] == "Geography"]

In [6]:
geo_to_id = {
  'Old Tor.': 6, 
  'York': 1, 
  'Scar.': 4, 
  'Etob.': 5, 
  'N York': 2, 
  'E York': 3,
}

banner1_geography = banner1_geography.assign(_id=lambda x: [geo_to_id[group] for group in x['group']])

In [7]:
merged_pd = banner1_geography.join(
  boundaries[["_id", "AREA_NAME", "geometry"]].set_index("_id"), 
  on="_id", 
  lsuffix="b"
)

In [8]:
merged_pd.columns

Index(['question_text', 'response', 'category', 'group', 'frequency', '_id',
       'AREA_NAME', 'geometry'],
      dtype='object')

In [9]:
merged_gdf = gpd.GeoDataFrame(merged_pd, geometry=merged_pd["geometry"])

In [10]:
# columns
# only want the nets
# show one question at a time?

In [11]:
merged_gdf = merged_gdf.assign(q_short=lambda x: [q.split(".")[0] for q in x['question_text']])

In [12]:
merged_gdf['q_short'].unique()

array(['Q1', 'Q2', 'Q4', 'Q5', 'Q6', 'Q7A', 'Q7B', 'Q7C', 'Q7D', 'Q7E',
       'Q7', 'Q8A', 'Q8B', 'Q8C', 'Q8D', 'Q8', 'Q9', 'Q10', 'Q11A',
       'Q11B', 'Q11C', 'Q12A', 'Q12B', 'Q12C', 'Q12D', 'Q12E', 'Q12F',
       'Q12', 'Q13A', 'Q13B', 'Q13C', 'Q13D', 'Q13E', 'Q13F', 'Q13G',
       'Q13H', 'Q13I', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19',
       'Q21A', 'Q21B', 'Q21C', 'Q22A', 'Q22B', 'Q22C', 'Q22D', 'Q23',
       'Q70A', 'Q70B', 'Q70C', 'Q70D', 'Q70', 'Q24A', 'Q24B', 'Q24C',
       'Q24', 'Q74AA', 'Q74AB', 'Q74AC', 'Q74AD', 'Q74AE', 'Q74BB',
       'Q74BD', 'Q25', 'Q25_COUNT', 'Q77', 'Q29', 'Q30', 'Q31', 'Q33',
       'Q35', 'Q35AA', 'Q35AB', 'Q35AC', 'Q35AD', 'Q35AE', 'Q35AF',
       'Q35AG', 'Q36', 'Q37', 'Q38', 'Q39A', 'Q39B', 'Q39C', 'Q39D',
       'Q68A', 'Q68B', 'Q68C', 'Q68D', 'Q68E', 'Q68F', 'Q68G', 'Q68H',
       'Q68I', 'Q68J', 'Q69', 'Q75', 'Q76A', 'Q76B', 'Q76C', 'Q76D',
       'Q41a', 'Q42', 'Q45', 'Q47', 'Q47AA', 'Q47AB', 'Q47AC', 'Q47AD',
       'Q47AE', 'Ch

In [39]:
felt_test = merged_gdf[merged_gdf["q_short"] == "Q38"].pivot(
  index=['_id'],
  columns=['response'],
  values=['frequency']
)
felt_test = felt_test['frequency'].join(
  boundaries[["_id", "AREA_NAME", "geometry"]].set_index("_id"), 
  on="_id", 
  lsuffix="b"
)

In [40]:
felt_test

,A bit stressful,Cannot say,Extremely stressful,NET: Not stressful -------------------,NET: Stressful ---------------,Not at all stressful,Not very stressful,Quite a bit stressful,AREA_NAME,geometry
_id,,,,,,,,,,
1,98.571167,5.126976,19.266890,68.850459,79.710020,18.743951,50.106508,60.443130,YORK,"MULTIPOLYGON (((-79.49262 43.64744, -79.49249 ..."
2,383.848479,18.335735,73.108666,349.094032,304.822599,92.360285,256.733747,231.713933,NORTH YORK,"MULTIPOLYGON (((-79.53584 43.70812, -79.53591 ..."
3,76.199361,0.000000,7.671625,78.675250,52.148269,19.481798,59.193452,44.476644,EAST YORK,"MULTIPOLYGON (((-79.28776 43.68978, -79.28909 ..."
4,358.808165,9.581309,70.041915,307.923149,264.558080,87.046277,220.876872,194.516165,SCARBOROUGH,"MULTIPOLYGON (((-79.28776 43.68978, -79.28792 ..."
5,209.264373,13.350424,30.868704,176.605086,159.735002,47.764104,128.840982,128.866298,ETOBICOKE,"MULTIPOLYGON (((-79.48165 43.61872, -79.48180 ..."
6,504.605934,10.003495,79.464435,310.732530,322.450086,85.433899,225.298631,242.985651,TORONTO,"MULTIPOLYGON (((-79.39652 43.62232, -79.39685 ..."


In [52]:
os.makedirs("question_tables", exist_ok=True)
felt_test_gdf = gpd.GeoDataFrame(felt_test, geometry=felt_test['geometry'])
felt_test_gdf.to_file(Path("question_tables") / "Q38.geojson", driver='GeoJSON')